# Aim

Aim makes it super easy to visualize and debug LangChain executions. Aim tracks inputs and outputs of LLMs and tools, as well as actions of agents.

With Aim, you can easily debug and examine an individual execution:

![](https://user-images.githubusercontent.com/13848158/227784778-06b806c7-74a1-4d15-ab85-9ece09b458aa.png)

Additionally, you have the option to compare multiple executions side by side:

![](https://user-images.githubusercontent.com/13848158/227784994-699b24b7-e69b-48f9-9ffa-e6a6142fd719.png)

Aim is fully open source, [learn more](https://github.com/aimhubio/aim) about Aim on GitHub.

Let's move forward and see how to enable and configure Aim callback.

<h3>Tracking LangChain Executions with Aim</h3>

In this notebook we will explore three usage scenarios. To start off, we will install the necessary packages and import certain modules. Subsequently, we will configure two environment variables that can be established either within the Python script or through the terminal.

In [1]:
%pip install --upgrade --quiet  aim
%pip install --upgrade --quiet  langchain
%pip install --upgrade --quiet  langchain-openai
%pip install --upgrade --quiet  google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 404.2 kB/s eta 0:00:00
   ━━━━━━━━━━

In [6]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [8]:
import os
from datetime import datetime
from langchain_community.callbacks.aim_callback import AimCallbackHandler
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_openai import OpenAI

Our examples use a GPT model as the LLM, and OpenAI offers an API for this purpose. You can obtain the key from the following link: https://platform.openai.com/account/api-keys .

We will use the SerpApi to retrieve search results from Google. To acquire the SerpApi key, please go to https://serpapi.com/manage-api-key .

In [9]:
os.environ["OPENAI_API_KEY"] = "---"
os.environ["SERPAPI_API_KEY"] = "---"

The event methods of `AimCallbackHandler` accept the LangChain module or agent as input and log at least the prompts and generated results, as well as the serialized version of the LangChain module, to the designated Aim run.

In [19]:
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1: OpenAI LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
llm = OpenAI(temperature=0, callbacks=callbacks)

The `flush_tracker` function is used to record LangChain assets on Aim. By default, the session is reset rather than being terminated outright.

<h3>Scenario 1</h3> In the first scenario, we will use OpenAI LLM.

In [11]:
# scenario 1 - LLM
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
aim_callback.flush_tracker(
    langchain_asset=llm,
    experiment_name="scenario 2: Chain with multiple SubChains on multiple generations",
)

<h3>Scenario 2</h3> Scenario two involves chaining with multiple SubChains across multiple generations.

In [12]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [13]:
# scenario 2 - Chain
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=callbacks)

test_prompts = [
    {
        "title": "documentary about good video games that push the boundary of game design"
    },
    {"title": "the phenomenon behind the remarkable speed of cheetahs"},
    {"title": "the best in class mlops tooling"},
]
synopsis_chain.apply(test_prompts)
aim_callback.flush_tracker(
    langchain_asset=synopsis_chain, experiment_name="scenario 3: Agent with Tools"
)

<ipython-input-13-191465f83bdd>:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=callbacks)




> Entering new LLMChain chain...
Prompt after formatting:
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: documentary about good video games that push the boundary of game design
Playwright: This is a synopsis for the above play:
Prompt after formatting:
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: the phenomenon behind the remarkable speed of cheetahs
Playwright: This is a synopsis for the above play:
Prompt after formatting:
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: the best in class mlops tooling
Playwright: This is a synopsis for the above play:



> Finished chain.


/usr/local/lib/python3.11/dist-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `literal['all']` but got `set` with value `set()` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `frozenset[str]` but got `set` with value `set()` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


<h3>Scenario 3</h3> The third scenario involves an agent with tools.

In [16]:
pip install langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.1 MB/s eta 0:00:00


In [29]:
from langgraph.prebuilt import create_react_agent
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

In [30]:
# scenario 3 - Agent with Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that answers questions accurately."),
    ("user", "{input}")
])

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Use the appropriate agent type
    verbose=True  # Set to True for debugging
)

# Define the input
inputs = {"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"}

# Run the agent
response = agent.run(inputs)

# Print the response
print(response)

# Flush the Aim callback tracker (if using Aim for tracking)
aim_callback = AimCallbackHandler()
aim_callback.flush_tracker(langchain_asset=agent, reset=False, finish=True)

<ipython-input-30-ec373c98c678>:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
<ipython-input-30-ec373c98c678>:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(inputs)




> Entering new AgentExecutor chain...
 I should use the search engine to find information about Leo DiCaprio's girlfriend.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio, 50, keeps a low profile in a face mask and baseball cap while his model girlfriend Vittoria Ceretti, 26, enjoys a glam night out at Frame dinner bash in Paris.
Thought: Now I need to use the calculator to raise her current age to the 0.43 power.
Action: Calculator
Action Input: 26 ^ 0.43
Observation: Answer: 4.059182145592686
Thought: I now know the final answer.
Final Answer: 4.059182145592686

> Finished chain.
4.059182145592686


/usr/local/lib/python3.11/dist-packages/pydantic/functional_validators.py:821: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `literal['all']` but got `set` with value `set()` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `frozenset[str]` but got `set` with value `set()` - serialized value may not be as expected
  function=lambda v, h: h(v), schema=original_schema
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_serializers.py:42: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `literal['all']` but got `set` with value `set()` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `frozenset[str]` but got `set` with value `set()` - serialized value may not be as expected
  v = handler(item, index)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Pydan